In [3]:
import os

import tensorflow as tf
import PIL
import numpy as np

os.environ["CUDA_VISIBLE_DEVICES"]="0"


In [4]:


IMAGE_SIZE = (2792,99)  # Images resize en taille ~ moyenne : moyenne pondérée
INPUT_IMAGE_SHAPE = (99,2792,1) # shape qui rentre dans le cnn
# même taille pour toutes les parties d'images, présentes en même densité

THRESHOLD_PREDICTION = 0.5

NUM_CALLS = tf.data.experimental.AUTOTUNE

DATA_PATH = "/home/cogrannr/roues/MEFRO/grises/"
DATA_PATH_DEFAUTS_REELS = "/home/cogrannr/roues/MEFRO/images_defauts/"
DATA_PATH_ROUES_ENTIERES="/home/cogrannr/roues/MEFRO/Images_Roues/Actuelle/gris_blanc/"

# Preprocessing

In [5]:
"""
    Fonction custom, car doit être fait en eager execution, sinon c'est un param "none" passé à Image.open
    ouvereture des photos / resize
"""
def open_image(file_name): 
    name = tf.get_static_value(file_name)
    name = name.decode()
    
    image = PIL.Image.open(name)
    
    image = image.resize(IMAGE_SIZE)
    image_arr = np.array(image)
    
    image_arr = image_arr[:,:,np.newaxis]
    return image_arr

def open_image_entiere(file_name): 
    name = tf.get_static_value(file_name)
    name = name.decode()
    
    image = PIL.Image.open(name)
    
    image = image.resize(IMAGE_SIZE)
    image_arr = np.array(image)
    return image_arr
    
def parse_images(filename,label): # appel "open_image"
    image_arr = tf.py_function(open_image,[filename],tf.float32)
    image_arr = tf.image.convert_image_dtype(image_arr,tf.float32)
    return image_arr,label

def parse_images_entieres(filename,label): # appel "open_image"
    image_arr = tf.py_function(open_image_entiere,[filename],tf.float32)
    image_arr = tf.image.convert_image_dtype(image_arr,tf.float32)
    return image_arr,label

def data_augmentation(image,label):
    image = tf.image.random_flip_left_right(image)
    #image = tf.image.random_brightness(image, max_delta=10.0/255.0)
    #image = tf.clip_by_value(image, 0.0, 1.0)

    return image,label
    
def reset_shapes_gray(image,label):
    image.set_shape(list(INPUT_IMAGE_SHAPE))
    return image, label


# Generate datasets

In [6]:
"""
    chargement des listes de fichiers photo
"""
def load_data(cover_dir,faulty_dir): 
    
    file_names_cover = tf.data.Dataset.list_files(cover_dir+"/*",shuffle=False)
    file_names_cover = file_names_cover.take(DATA_SIZE//2)
    labels_0 = tf.data.Dataset.from_tensors(0).repeat()
    data_set_cover = tf.data.Dataset.zip((file_names_cover,labels_0))
    
    file_names_faulty = tf.data.Dataset.list_files(faulty_dir+"/*",shuffle=False)
    file_names_faulty = file_names_faulty.take(DATA_SIZE//2)
    labels_1 = tf.data.Dataset.from_tensors(1).repeat()
    data_set_faulty = tf.data.Dataset.zip((file_names_faulty,labels_1))
   
    data_set = data_set_cover.concatenate(data_set_faulty)
    
    tf.random.set_seed(1)
    data_set = data_set.shuffle(DATA_SIZE)
   
    return data_set

def load_data_entieres(cover_dir,faulty_dir, num_negative, num_positive): 
    
    file_names_cover = tf.data.Dataset.list_files(cover_dir,shuffle=False)
    labels_0 = tf.data.Dataset.from_tensors(0).repeat()
    
    data_set_cover = tf.data.Dataset.zip((file_names_cover,labels_0))
    data_set_cover = data_set_cover.take(num_negative)
    
    
    file_names_faulty = tf.data.Dataset.list_files(faulty_dir,shuffle=False)
    labels_1 = tf.data.Dataset.from_tensors(1).repeat()
    
    data_set_faulty = tf.data.Dataset.zip((file_names_faulty,labels_1))
    data_set_faulty = data_set_faulty.take(num_positive)
    
    data_set = data_set_cover.concatenate(data_set_faulty)
    
    tf.random.set_seed(1)
    data_set = data_set.shuffle(DATA_SIZE)
   
    return data_set


""" 
    répartition des fichiers enn ensembles train/valid
"""
def generate_train_valid_test(cover_dir,faulty_dir, augment_data=False, entieres=False,
                             num_positive=0,num_negative=0): 
    
    if entieres==True:
        data_set = load_data_entieres(cover_dir, faulty_dir, num_negative,num_positive)
    else:
        data_set = load_data(cover_dir, faulty_dir)
        
    train_dataset = data_set.take(DATA_TRAIN_SIZE)
    valid_dataset = data_set.skip(DATA_TRAIN_SIZE).take(DATA_VALID_SIZE)
    test_dataset = data_set.skip(DATA_TRAIN_SIZE+DATA_VALID_SIZE).take(DATA_TEST_SIZE)

    
    datasets_list = []
    
    for data_set in [train_dataset, valid_dataset, test_dataset]:

        if data_set == train_dataset:
            if entieres == True:
                data_set = data_set.map(parse_images_entieres, num_parallel_calls=NUM_CALLS)
            else:
                data_set = data_set.map(parse_images, num_parallel_calls=NUM_CALLS)
            if augment_data == True:
                data_set = data_set.map(reset_shapes_gray, num_parallel_calls=NUM_CALLS)
                data_set = data_set.map(data_augmentation, num_parallel_calls=NUM_CALLS)
        else:
            if entieres == True:
                data_set = data_set.map(parse_images_entieres, num_parallel_calls=NUM_CALLS)
            else:
                data_set = data_set.map(parse_images, num_parallel_calls=NUM_CALLS)
            
        data_set = data_set.map(reset_shapes_gray, num_parallel_calls=NUM_CALLS)
            
        data_set = data_set.batch(BATCH_SIZE)
        data_set = data_set.prefetch(1)
        
        datasets_list.append(data_set)
        
    return datasets_list

In [7]:
# confusion matrix : 

def confusion_matrix(test_dataset,model, num_positive_class, num_negative_class):
    
    prediction = np.array([])
    labels = tf.constant([], dtype=tf.int32)

    for (data_batch, label_batch) in test_dataset:
        prediction = np.concatenate((prediction, model.predict_on_batch(data_batch)[:,0]))
        labels = tf.concat((labels, label_batch), axis=0)
  
    prediction[prediction > THRESHOLD_PREDICTION] = 1
    prediction[prediction <= THRESHOLD_PREDICTION] = 0
        
    confusion_matrix = tf.math.confusion_matrix(
        labels=labels,predictions=tf.convert_to_tensor(prediction),num_classes=2)

    l = tf.get_static_value(labels)
    
    num_pos = np.sum(l==1)
    num_neg = np.sum(l==0)
    print("num pos ",num_pos)
    print("num neg ",num_neg)
    
    nomralized_matrix = tf.get_static_value(confusion_matrix)
    nomralized_matrix = nomralized_matrix.astype(np.float32)
    
    nomralized_matrix[0,:] = nomralized_matrix[0,:]/num_neg
    nomralized_matrix[1,:] = nomralized_matrix[1,:]/num_pos

    print("\nConfusion matrix : ")
    print(tf.get_static_value(confusion_matrix))
    print("\nNormalized confusion matrix : ")
    print(nomralized_matrix)

 

# Model architecture

In [8]:
def init_model(architecture=1):
    
    if architecture == 1:    # 1M param : MARCHE OK
        model_input = tf.keras.Input(shape=INPUT_IMAGE_SHAPE)
        conv_1 = tf.keras.layers.Conv2D(32,(3,3),activation='relu',padding='SAME',strides=(1,2))(model_input)
        conv_2 = tf.keras.layers.Conv2D(32,(3,3),activation='relu',padding='SAME',strides=(1,1))(conv_1)
        bn_1 = tf.keras.layers.BatchNormalization()(conv_2)
        mxp_1 = tf.keras.layers.MaxPool2D(pool_size=(2,2))(bn_1)
        
        conv_3 = tf.keras.layers.Conv2D(64,(3,3),activation='relu',padding='SAME',strides=(1,2))(mxp_1)
        conv_4 = tf.keras.layers.Conv2D(64,(3,3),activation='relu',padding='SAME',strides=(1,1))(conv_3)
        bn_2 = tf.keras.layers.BatchNormalization()(conv_4)
        mxp_2 = tf.keras.layers.MaxPool2D(pool_size=(2,2))(bn_2)
        
        conv_5 = tf.keras.layers.Conv2D(128,(3,3),activation='relu',padding='SAME',strides=(1,2))(mxp_2)
        conv_6 = tf.keras.layers.Conv2D(128,(3,3),activation='relu',padding='SAME',strides=(1,1))(conv_5)
        bn_3 = tf.keras.layers.BatchNormalization()(conv_6)
        mxp_3 = tf.keras.layers.MaxPool2D(pool_size=(2,2))(bn_3)
        
        conv_7 = tf.keras.layers.Conv2D(256,(3,3),activation='relu',padding='SAME',strides=(1,2))(mxp_3)
        conv_8 = tf.keras.layers.Conv2D(256,(3,3),activation='relu',padding='SAME',strides=(1,1))(conv_7)
        bn_4 = tf.keras.layers.BatchNormalization()(conv_8)
        mxp_4 = tf.keras.layers.MaxPool2D(pool_size=(2,2))(bn_4)
        
        flat = tf.keras.layers.Flatten()(mxp_4)
        drop = tf.keras.layers.Dropout(0.3)(flat)
        out = tf.keras.layers.Dense(1,activation="sigmoid")(drop)
    
    if architecture == 2:   # 300 K parametres
        model_input = tf.keras.Input(shape=INPUT_IMAGE_SHAPE)
        conv_1 = tf.keras.layers.Conv2D(32,(3,3),activation='relu',padding='SAME',strides=(1,2))(model_input)
        conv_2 = tf.keras.layers.Conv2D(32,(3,3),activation='relu',padding='SAME',strides=(1,2))(conv_1)
        bn_1 = tf.keras.layers.BatchNormalization()(conv_2)
        
        mxp_1 = tf.keras.layers.MaxPool2D(pool_size=(2,2))(bn_1)
        conv_3 = tf.keras.layers.Conv2D(64,(3,3),activation='relu',padding='SAME',strides=(1,2))(mxp_1)
        conv_4 = tf.keras.layers.Conv2D(64,(3,3),activation='relu',padding='SAME',strides=(1,2))(conv_3)
        bn_2 = tf.keras.layers.BatchNormalization()(conv_4)
        
        mxp_2 = tf.keras.layers.MaxPool2D(pool_size=(2,2))(bn_2)
        conv_5 = tf.keras.layers.Conv2D(128,(3,3),activation='relu',padding='SAME',strides=(1,2))(mxp_2)
        conv_6 = tf.keras.layers.Conv2D(128,(3,3),activation='relu',padding='SAME',strides=(1,2))(conv_5)
        #conv_7 = tf.keras.layers.Conv2D(64,(1,1),activation="relu")(conv_6)
        
        flat = tf.keras.layers.Flatten()(conv_6)
        drop_1  = tf.keras.layers.Dropout(rate = 0.3)(flat)
        out = tf.keras.layers.Dense(1,activation="sigmoid")(drop_1)

    model = tf.keras.Model(inputs=model_input, outputs=out)
    print(model.summary())
    
    return model

# Training settings 

In [9]:
def train_model(model,train_dataset,valid_dataset, check_point_name, num_epochs,tensor_board_name=""):
    
    precision_metric = tf.keras.metrics.Precision(thresholds=THRESHOLD_PREDICTION)
    
    call_backs_list = [
        #tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", patience = 2),
        tf.keras.callbacks.ModelCheckpoint(
            filepath =check_point_name, monitor ="val_accuracy", save_best_only = True),
    ]
    
    if tensor_board_name != "":
        board = tf.keras.callbacks.TensorBoard(log_dir='./tensor_board_'+tensor_board_name)
        call_backs_list.append(board)
   
    optim = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
  
    model.compile(loss='binary_crossentropy',optimizer=optim,
        metrics=[precision_metric,"accuracy"]) 
    
    model.fit(train_dataset,epochs=num_epochs,
              validation_data=valid_dataset,
             callbacks=call_backs_list
             )
    
    return model

# Train Images gray


In [15]:
DATA_SIZE = int(15e3) # taille des données (test/valid/test compris), max : 146K
DATA_TRAIN_SIZE = int(DATA_SIZE*0.6)
DATA_VALID_SIZE = int(DATA_SIZE*0.2)
DATA_TEST_SIZE = int(DATA_SIZE*0.2)


LEARNING_RATE = 0.0001 

NUM_EPOCHS = {"galbe":10, "ajours":10, "crochet":10, "central":150}

SAVE_PATH = "./trained_models/wheel_part/gray/"

NUM_POSITIVE_CLASS = DATA_TEST_SIZE//2
NUM_NEGATIVE_CLASS = DATA_TEST_SIZE//2

BATCH_SIZE = 4

MODEL_NAME = "model_v4_gray_"

In [10]:
mirrored_strategy = tf.distribute.MirroredStrategy()

with mirrored_strategy.scope():

    for wheel_part in ["galbe", "ajours", "crochet", "central"]:
        train_dataset, valid_dataset, test_dataset = generate_train_valid_test(
            DATA_PATH+"img_"+wheel_part, DATA_PATH+"img_"+wheel_part+"_avec_defauts")

        model = init_model(architecture=1)
        model = train_model(model, train_dataset, valid_dataset,
                        check_point_name=SAVE_PATH+MODEL_NAME+wheel_part+".h5", num_epochs=NUM_EPOCHS[wheel_part])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/

KeyboardInterrupt: 

In [ ]:
# pour la RAM : bien shutdown le notebook avant cette partie !

for wheel_part in ["galbe", "ajours", "crochet", "central"]:
    
    train_dataset, valid_dataset, test_dataset = generate_train_valid_test(
            DATA_PATH+"img_"+wheel_part, DATA_PATH+"img_"+wheel_part+"_avec_defauts")
    
    check_point = tf.keras.models.load_model(SAVE_PATH+MODEL_NAME+wheel_part+".h5")
    
    print("evaluating resnet model for wheel part : "+wheel_part)
    check_point.evaluate(test_dataset, verbose=1)
    
    confusion_matrix(test_dataset, check_point, NUM_POSITIVE_CLASS, NUM_NEGATIVE_CLASS)
    

evaluating resnet model for wheel part : galbe


In [11]:
for name in ["galbe", "ajours", "crochet", "central"]:
    check_point = tf.keras.models.load_model(SAVE_PATH+MODEL_NAME+name+".h5")
    print("model ",name," loaded")
    for wheel_part in ["galbe", "ajours", "crochet", "central"]:
    
        train_dataset, valid_dataset, test_dataset = generate_train_valid_test(
            DATA_PATH+"img_"+wheel_part, DATA_PATH+"img_"+wheel_part+"_avec_defauts")
        print("evaluate model",name," on data : ",wheel_part)
        check_point.evaluate(test_dataset, verbose=1)


model  galbe  loaded
evaluate model galbe  on data :  galbe
750/750 [==============================] - 51s 68ms/step - loss: 0.7036 - precision: 0.5053 - accuracy: 0.5053
evaluate model galbe  on data :  ajours
750/750 [==============================] - 51s 68ms/step - loss: 0.6933 - precision: 0.5089 - accuracy: 0.5063
evaluate model galbe  on data :  crochet
750/750 [==============================] - 52s 69ms/step - loss: 0.7142 - precision: 0.5053 - accuracy: 0.5053
evaluate model galbe  on data :  central
750/750 [==============================] - 51s 68ms/step - loss: 0.8143 - precision: 0.5053 - accuracy: 0.5053
model  ajours  loaded
evaluate model ajours  on data :  galbe
750/750 [==============================] - 52s 69ms/step - loss: 0.3976 - precision_3: 0.9961 - accuracy: 0.91630s - loss: 0.4030 - precision_3: 0
evaluate model ajours  on data :  ajours
750/750 [==============================] - 52s 69ms/step - loss: 0.0220 - precision_3: 1.0000 - accuracy: 0.9920
evaluate mo

# Entrainement residus

In [10]:
DATA_SIZE = int(5e3) # taille des données (test/valid/test compris), max : 146K
DATA_TRAIN_SIZE = int(DATA_SIZE*0.6)
DATA_VALID_SIZE = int(DATA_SIZE*0.2)
DATA_TEST_SIZE = int(DATA_SIZE*0.2)

LEARNING_RATE = 0.0001 

NUM_EPOCHS = 4

SAVE_PATH = "./trained_models/wheel_part/residus/"

NUM_POSITIVE_CLASS = DATA_TEST_SIZE//2
NUM_NEGATIVE_CLASS = DATA_TEST_SIZE//2

BATCH_SIZE = 4

MODEL_NAME = "model_v4_residus_"

In [ ]:
mirrored_strategy = tf.distribute.MirroredStrategy()

with mirrored_strategy.scope():

    for wheel_part in ["galbe", "ajours", "crochet", "central"]:
        train_dataset, valid_dataset, test_dataset = generate_train_valid_test(
            DATA_PATH+"img_"+wheel_part+"_residus", DATA_PATH+"img_"+wheel_part+"_residus_defauts")

        model = init_model(architecture=1)
        model = train_model(model, train_dataset, valid_dataset,
                check_point_name=SAVE_PATH+MODEL_NAME+wheel_part+".h5", num_epochs=NUM_EPOCHS)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2')
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 99, 2792, 1)]     0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 99, 1396, 32)      320       
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 99, 1396, 32)      9248      
_________________________________________________________________
batch_normalization_4 (Batch (None, 99, 1396, 32)      128       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 49, 698, 32)       0         
____________________________________________________________

In [19]:
# pour la RAM : bien shutdown le notebook avant cette partie !

for wheel_part in ["galbe", "ajours", "crochet", "central"]:
    
    train_dataset, valid_dataset, test_dataset = generate_train_valid_test(
            DATA_PATH+"img_"+wheel_part+"_residus", DATA_PATH+"img_"+wheel_part+"_residus_defauts")
    
    check_point = tf.keras.models.load_model(SAVE_PATH+MODEL_NAME+wheel_part+".h5")
    
    print("evaluating resnet model for wheel part : "+wheel_part)
    
    check_point.evaluate(test_dataset, verbose=1)
    
    confusion_matrix(test_dataset, check_point, NUM_POSITIVE_CLASS, NUM_NEGATIVE_CLASS)
    

evaluating resnet model for wheel part : galbe
1825/1825 [==============================] - 88s 48ms/step - loss: 0.0474 - precision_1: 0.9921 - accuracy: 0.9876
num pos  14540
num neg  14660

Confusion matrix : 
[[14550   110]
 [  245 14295]]

Normalized confusion matrix : 
[[0.9924966  0.00750341]
 [0.01685007 0.98314995]]
evaluating resnet model for wheel part : ajours
1825/1825 [==============================] - 90s 50ms/step - loss: 0.0822 - precision_2: 0.9899 - accuracy: 0.9785
num pos  14540
num neg  14660

Confusion matrix : 
[[14532   128]
 [  474 14066]]

Normalized confusion matrix : 
[[0.99126875 0.00873124]
 [0.03259972 0.96740025]]
evaluating resnet model for wheel part : crochet
1825/1825 [==============================] - 117s 64ms/step - loss: 0.0882 - precision_3: 0.9947 - accuracy: 0.9726
num pos  14540
num neg  14660

Confusion matrix : 
[[14577    83]
 [  688 13852]]

Normalized confusion matrix : 
[[0.99433833 0.00566166]
 [0.04731774 0.95268226]]
evaluating resn

# Test défauts réels images gray

In [20]:
# liste des sizes de chaque partie de roues
DATA_SIZE_TEST = { "galbe":1048 , "ajours":192 , "crochet":1792 , "central":None }  

SAVE_PATH = "./trained_models/wheel_part/gray/"

MODEL_NAME = "model_v4_gray_"

for wheel_part in ["galbe", "ajours", "crochet"]:   #, "central"]:
    
    DATA_SIZE = DATA_SIZE_TEST[wheel_part]
    
    NUM_POSITIVE_CLASS = DATA_SIZE//2
    NUM_NEGATIVE_CLASS = DATA_SIZE//2
    
    test_dataset = load_data(DATA_PATH+"img_"+wheel_part, 
            DATA_PATH_DEFAUTS_REELS+"img_"+wheel_part+"_avec_defauts")
    
    test_dataset = test_dataset.map(parse_images, num_parallel_calls=NUM_CALLS)
    test_dataset = test_dataset.map(reset_shapes_gray, num_parallel_calls=NUM_CALLS)

    test_dataset = test_dataset.batch(BATCH_SIZE)
    test_dataset = test_dataset.prefetch(1)
  
    check_point = tf.keras.models.load_model(SAVE_PATH+MODEL_NAME+wheel_part+".h5")
    print("evaluating model for wheel part : "+wheel_part)
    check_point.evaluate(test_dataset, verbose=1)
    
    confusion_matrix(test_dataset, check_point, NUM_POSITIVE_CLASS, NUM_NEGATIVE_CLASS)
    

evaluating model for wheel part : galbe
66/66 [==============================] - 3s 53ms/step - loss: 0.3961 - precision_2: 0.9947 - accuracy: 0.8540
num pos  524
num neg  524

Confusion matrix : 
[[522   2]
 [151 373]]

Normalized confusion matrix : 
[[0.9961832  0.00381679]
 [0.28816795 0.71183205]]
evaluating model for wheel part : ajours
12/12 [==============================] - 1s 73ms/step - loss: 0.8374 - precision_3: 1.0000 - accuracy: 0.7969
num pos  96
num neg  96

Confusion matrix : 
[[96  0]
 [39 57]]

Normalized confusion matrix : 
[[1.      0.     ]
 [0.40625 0.59375]]
evaluating model for wheel part : crochet
112/112 [==============================] - 7s 66ms/step - loss: 0.3743 - precision_4: 0.9893 - accuracy: 0.8560
num pos  896
num neg  896

Confusion matrix : 
[[889   7]
 [251 645]]

Normalized confusion matrix : 
[[0.9921875  0.0078125 ]
 [0.28013393 0.7198661 ]]


# Test défauts réels residus


In [21]:
DATA_SIZE_TEST = { "galbe":1048 , "ajours":192 , "crochet":1792 , "central":None }  

SAVE_PATH = "./trained_models/wheel_part/residus/"

MODEL_NAME = "model_v4_residus_"

for wheel_part in ["galbe", "ajours", "crochet"]:   #, "central"]:
    
    DATA_SIZE = DATA_SIZE_TEST[wheel_part]
    
    NUM_POSITIVE_CLASS = DATA_SIZE//2
    NUM_NEGATIVE_CLASS = DATA_SIZE//2
    
    test_dataset = load_data(DATA_PATH+"img_"+wheel_part+"_residus", 
            DATA_PATH_DEFAUTS_REELS+"img_"+wheel_part+"_residus_defauts")
    
    test_dataset = test_dataset.map(parse_images, num_parallel_calls=NUM_CALLS)
    test_dataset = test_dataset.map(reset_shapes_gray, num_parallel_calls=NUM_CALLS)

    test_dataset = test_dataset.batch(BATCH_SIZE)
    test_dataset = test_dataset.prefetch(1)
  
    check_point = tf.keras.models.load_model(SAVE_PATH+MODEL_NAME+wheel_part+".h5")
    print("evaluating model for wheel part : "+wheel_part)
    check_point.evaluate(test_dataset, verbose=1)
    
    confusion_matrix(test_dataset, check_point, NUM_POSITIVE_CLASS, NUM_NEGATIVE_CLASS)

evaluating model for wheel part : galbe
66/66 [==============================] - 4s 54ms/step - loss: 0.2337 - precision_1: 0.9866 - accuracy: 0.9160
num pos  524
num neg  524

Confusion matrix : 
[[518   6]
 [ 82 442]]

Normalized confusion matrix : 
[[0.9885496  0.01145038]
 [0.15648855 0.84351146]]
evaluating model for wheel part : ajours
12/12 [==============================] - 1s 62ms/step - loss: 0.3024 - precision_2: 1.0000 - accuracy: 0.8281
num pos  96
num neg  96

Confusion matrix : 
[[96  0]
 [33 63]]

Normalized confusion matrix : 
[[1.      0.     ]
 [0.34375 0.65625]]
evaluating model for wheel part : crochet
112/112 [==============================] - 7s 58ms/step - loss: 0.1517 - precision_3: 0.9899 - accuracy: 0.9347
num pos  896
num neg  896

Confusion matrix : 
[[888   8]
 [109 787]]

Normalized confusion matrix : 
[[0.9910714  0.00892857]
 [0.12165178 0.87834823]]


# Train roue entiere : (roues direct)


In [8]:
LEARNING_RATE = 0.0001
NUM_EPOCHS = 4

IMAGE_SIZE = (1624,1234)  # Images resize en taille ~ moyenne : moyenne pondérée
INPUT_IMAGE_SHAPE = (1234,1624,3) # shape qui rentre dans le cnn
    #roues deja en rgb

BATCH_SIZE = 6

DATA_SIZE = int(74533) # taille des données (test/valid/test compris), max : 74533;    6845 : rapport 1:4
DATA_TRAIN_SIZE = int(DATA_SIZE*0.6)
DATA_VALID_SIZE = int(DATA_SIZE*0.2)
DATA_TEST_SIZE = int(DATA_SIZE*0.2)

#1369 : total de positives

NUM_POSITIVE_CLASS_TOTAL = 1369 # constant
NUM_NEGATIVE_CLASS_TOTAL = DATA_SIZE - NUM_POSITIVE_CLASS_TOTAL

NUM_POSITIVE_CLASS = int(NUM_POSITIVE_CLASS_TOTAL*0.2) #273 constant

NUM_NEGATIVE_CLASS = int(NUM_NEGATIVE_CLASS_TOTAL*0.2) # 1095 pour 1:4    ;  pour tout 

SAVE_PATH = "./trained_models/whole_wheel/"
MODEL_NAME_1 = "model_v4_whole_wheel_1_pour_4.h5"   #"model_v4_whole_wheel_1_pour_4.h5"  1:4
MODEL_NAME_2 = "model_v4_whole_wheel_1_pour_70.h5"

In [ ]:

mirrored_strategy = tf.distribute.MirroredStrategy() 

with mirrored_strategy.scope():

    train_dataset, valid_dataset, test_dataset = generate_train_valid_test(
        cover_dir=DATA_PATH_ROUES_ENTIERES+"*/*OK*", 
        faulty_dir=(DATA_PATH_ROUES_ENTIERES+"*/*DJ*",DATA_PATH_ROUES_ENTIERES+"*/*DG*"),
        augment_data=False,entieres=True, 
        num_positive=NUM_POSITIVE_CLASS_TOTAL, num_negative=NUM_NEGATIVE_CLASS_TOTAL)

    model = init_model(architecture=1)
    model = train_model(model, train_dataset, valid_dataset,
                                  check_point_name=SAVE_PATH+MODEL_NAME_2, num_epochs=NUM_EPOCHS)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2')
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localho

In [10]:
# test sur defauts fabriqués, modele 1

train_dataset, valid_dataset, test_dataset = generate_train_valid_test(
    cover_dir=DATA_PATH_ROUES_ENTIERES+"*/*OK*", 
    faulty_dir=(DATA_PATH_ROUES_ENTIERES+"*/*DJ*",DATA_PATH_ROUES_ENTIERES+"*/*DG*"),
    augment_data=False,entieres=True, num_positive=NUM_POSITIVE_CLASS_TOTAL, num_negative=NUM_NEGATIVE_CLASS_TOTAL)


check_point = tf.keras.models.load_model(SAVE_PATH+MODEL_NAME_1)
check_point.evaluate(test_dataset, verbose=1)

confusion_matrix(test_dataset, check_point, NUM_POSITIVE_CLASS, NUM_NEGATIVE_CLASS)

229/229 [==============================] - 33s 145ms/step - loss: 0.0774 - precision: 1.0000 - accuracy: 0.9971
num pos  264
num neg  1105

Confusion matrix : 
[[1105    0]
 [   5  259]]

Normalized confusion matrix : 
[[1.         0.        ]
 [0.01893939 0.9810606 ]]


In [28]:

# defauts fabriqués, modele 2
train_dataset, valid_dataset, test_dataset = generate_train_valid_test(
    cover_dir=DATA_PATH_ROUES_ENTIERES+"*/*OK*", 
    faulty_dir=(DATA_PATH_ROUES_ENTIERES+"*/*DJ*",DATA_PATH_ROUES_ENTIERES+"*/*DG*"),
    augment_data=False,entieres=True, num_positive=NUM_POSITIVE_CLASS_TOTAL, num_negative=NUM_NEGATIVE_CLASS_TOTAL)


check_point = tf.keras.models.load_model(SAVE_PATH+MODEL_NAME_2)
check_point.evaluate(test_dataset, verbose=1)



confusion_matrix(test_dataset, check_point, NUM_POSITIVE_CLASS, NUM_NEGATIVE_CLASS)

1656/1656 [==============================] - 1078s 651ms/step - loss: 0.0605 - precision_1: 0.7908 - accuracy: 0.9942
num pos  280
num neg  14616

Confusion matrix : 
[[14549    67]
 [   21   259]]

Normalized confusion matrix : 
[[0.995416   0.00458402]
 [0.075      0.925     ]]
